In [ ]:
!pip install dash

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd



In [ ]:
df_ = pd.read_csv("finaldf.csv")

In [ ]:


# Features for plotting
plot_features = ['PointsRedeemedRatio', 'FlightsWithCompanionsRatio', 'AvgDistancePerFlight',
                 'KMPerRedeemedPoint', 'UnredeemedPoints', 'AvgMonthlyFlights']
categorical_filters = ['Education', 'Income', 'Province or State']

df_dash = df_.copy()
df_dash['Cluster'] = df_['Merged_Clusters_2'].astype(str)  # for colors

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Interactive Cluster Visualization Dashboard"),
    
    html.Div([
        html.Label("Filter by Education"),
        dcc.Dropdown(
            id='filter-education',
            options=[{'label': edu, 'value': edu} for edu in sorted(df_dash['Education'].unique())],
            value=None,
            multi=True,
            placeholder="Select Education..."
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': 10}),
    
    html.Div([
        html.Label("Filter by Income"),
        dcc.Dropdown(
            id='filter-income',
            options=[{'label': inc, 'value': inc} for inc in sorted(df_dash['Income'].unique())],
            value=None,
            multi=True,
            placeholder="Select Income..."
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': 10}),
    
    dcc.Graph(id='cluster-3d-plot')
])

@app.callback(
    Output('cluster-3d-plot', 'figure'),
    Input('filter-education', 'value'),
    Input('filter-income', 'value')
)
def update_3d_plot(selected_edu, selected_income):
    filtered_df = df_dash.copy()
    if selected_edu:
        filtered_df = filtered_df[filtered_df['Education'].isin(selected_edu)]
    if selected_income:
        filtered_df = filtered_df[filtered_df['Income'].isin(selected_income)]
    
    fig = px.scatter_3d(
        filtered_df,
        x=plot_features[0],
        y=plot_features[1],
        z=plot_features[2],
        color='Cluster',
        hover_data=plot_features + categorical_filters,
        opacity=0.7,
        size_max=8
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis_title=plot_features[0],
            yaxis_title=plot_features[1],
            zaxis_title=plot_features[2]
        )
    )
    
    return fig

if __name__ == '__main__':
    app.run(debug=True)
